In [0]:
from pyspark.sql.functions import sum, desc, col, avg, round
from pyspark.sql import Window

In [0]:
major_incident_df = spark.read \
    .table("mta_silver.fct_major_incident")

dim_line_df = spark.read \
    .table("mta_silver.dim_line")

date_df = spark.read \
    .table("mta_silver.dim_date")

In [0]:
window_year = Window.partitionBy("dte_year")

yr_incidents_by_line_t1_df = major_incident_df \
    .join(dim_line_df, major_incident_df.lin_sk == dim_line_df.lin_sk, "inner") \
    .join(date_df, major_incident_df.dte_sk == date_df.dte_sk, "inner") \
    .withColumn("yr_total_incidents", sum(col("inc_count")).over(window_year)) \
    .select(
        "lin_nk",
        "lin_name",
        "lin_type",
        "dte_year",
        "inc_count",
        "yr_total_incidents"
    ).groupBy(
        "lin_nk",
        "lin_name",
        "lin_type",
        "dte_year",
        "yr_total_incidents",
    ).agg(sum("inc_count").alias("yr_incidents")) \
    .orderBy("dte_year", desc("yr_incidents"))

yr_incidents_by_line_t2_df = yr_incidents_by_line_t1_df \
    .withColumn("yr_avg_incidents", round(avg("yr_incidents").over(window_year), 2))

In [0]:
yr_incidents_by_line_final_df = yr_incidents_by_line_t2_df.select(
    col("lin_nk").alias("yil_nk"),
    col("lin_name").alias("yil_name"),
    col("lin_type").alias("yil_type"),
    col("dte_year").alias("yil_year"),
    col("yr_incidents").alias("yil_incidents"),
    col("yr_avg_incidents").alias("yil_avg_yearly_incidents_per_line"),
    col("yr_total_incidents").alias("yil_total_yearly_incidents")
)

In [0]:
yr_incidents_by_line_final_df.write.format("delta").mode("overwrite").option("mergeSchema", "true").saveAsTable("mta_gold.rpt_year_incidents_by_line")

In [0]:
%sql
SELECT * FROM mta_gold.rpt_year_incidents_by_line;

yil_nk,yil_name,yil_type,yil_year,yil_incidents,yil_avg_yearly_incidents_per_line,yil_total_yearly_incidents
6,Lexington Avenue Local,Local,2015,87,40.05,881
2,Seventh Avenue Express,Express,2015,72,40.05,881
F,Sixth Avenue Local,Local,2015,68,40.05,881
5,Lexington Avenue Express,Express,2015,66,40.05,881
E,Eighth Avenue Local,Local,2015,63,40.05,881
4,Lexington Avenue Express,Express,2015,60,40.05,881
R,Broadway Local,Local,2015,58,40.05,881
N,Broadway Express,Express,2015,49,40.05,881
3,Seventh Avenue Express,Express,2015,43,40.05,881
A,Eighth Avenue Express,Express,2015,43,40.05,881


In [0]:
dbutils.notebook.exit("Success")